# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd

database = pd.read_csv("/Users/miguelmoreno/Desktop/IRONHACK/week_13_day_3/lab-neural-networks/your-code/tic-tac-toe.csv")
database.head()


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [2]:
database.describe()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,True
freq,418,378,418,378,458,378,418,378,418,626


In [10]:
database.sample(5)

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
940,b,b,o,b,x,o,x,x,o,False
584,b,o,b,o,o,x,x,x,x,True
192,x,o,b,x,x,o,o,b,x,True
915,b,o,x,x,o,x,b,o,b,False
690,x,o,o,x,x,o,b,x,o,False


In [14]:
database.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

In [15]:
database.isnull().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [19]:
database.info

<bound method DataFrame.info of     TL TM TR ML MM MR BL BM BR  class
0    x  x  x  x  o  o  x  o  o   True
1    x  x  x  x  o  o  o  x  o   True
2    x  x  x  x  o  o  o  o  x   True
3    x  x  x  x  o  o  o  b  b   True
4    x  x  x  x  o  o  b  o  b   True
..  .. .. .. .. .. .. .. .. ..    ...
953  o  x  x  x  o  o  o  x  x  False
954  o  x  o  x  x  o  x  o  x  False
955  o  x  o  x  o  x  x  o  x  False
956  o  x  o  o  x  x  x  o  x  False
957  o  o  x  x  x  o  o  x  x  False

[958 rows x 10 columns]>

In [20]:
database['class'].value_counts()

class
True     626
False    332
Name: count, dtype: int64

In [23]:
for col in database.columns[:-1]:
    print(f"\nColumna {col}:")
    print(database[col].value_counts())


Columna TL:
TL
x    418
o    335
b    205
Name: count, dtype: int64

Columna TM:
TM
x    378
o    330
b    250
Name: count, dtype: int64

Columna TR:
TR
x    418
o    335
b    205
Name: count, dtype: int64

Columna ML:
ML
x    378
o    330
b    250
Name: count, dtype: int64

Columna MM:
MM
x    458
o    340
b    160
Name: count, dtype: int64

Columna MR:
MR
x    378
o    330
b    250
Name: count, dtype: int64

Columna BL:
BL
x    418
o    335
b    205
Name: count, dtype: int64

Columna BM:
BM
x    378
o    330
b    250
Name: count, dtype: int64

Columna BR:
BR
x    418
o    335
b    205
Name: count, dtype: int64


In [24]:
print(f"Número de registros duplicados: {database.duplicated().sum()}")

Número de registros duplicados: 0


In [21]:
#958 rows and 10 columns
#fist 9 are objects last one is boolean
#not null values
#3 unique values in the first 9 columns "x", "o", "b"
#Player X won 626 partidas y perdio o empato 332. Esto quiere decir que no esta perfectamente balanceado y puede afectar el aprendizaje del modelo.


In [26]:
#REEMPLAZAR LOS VALORES A NUMEROS PARA QUE SE PUEDAN LEER POR EL MODELO 
#PRIMERO SE CREA UN MAPEO DE LOS DATOS PARA CAMBIARLOS 

mapping = {'x':1, 'o':-1, 'b':0}

#SE APLICA EL MAPEO A TODAS LAS COLUMNAS EXCEPTO LA ULTIMA (CLASS)
X = database.iloc[:, :-1].replace(mapping)
#SE REALIZA LO MISMO PERO AHORA CON LA ULTIMA COLUMNA (CLASS)
y = database['class'].astype(int)

#ESTO AUTOMATICAMENTE TAMBIEN NOS AYUDA A SEPARAR LAS VARIABLES DE ENTRADA Y DE SALIDA DE LA RED NEURONAL.
print(X.head())
print(y.head())

   TL  TM  TR  ML  MM  MR  BL  BM  BR
0   1   1   1   1  -1  -1   1  -1  -1
1   1   1   1   1  -1  -1  -1   1  -1
2   1   1   1   1  -1  -1  -1  -1   1
3   1   1   1   1  -1  -1  -1   0   0
4   1   1   1   1  -1  -1   0  -1   0
0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64


/var/folders/tc/dvbccp5x0rzg3_4x3qdj46zh0000gn/T/ipykernel_73969/743822477.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = database.iloc[:, :-1].replace(mapping)


In [27]:
#NORMALIZAR LOS VALORES DE ENTRADA

#Min-Max Scaling: Escala los datos entre 0 y 1: Cuando los datos tienen rangos conocidos y acotados
#Standard Scaling (Z-score): Convierte los datos a media 0 y desviación estándar 1: Si quieres datos centrados alrededor de 0 (muy común en ML)

from sklearn.preprocessing import StandardScaler
#CREAR EL NORMALIZADOR
scaler = StandardScaler()
#AJUSTAR Y TRANSFORMAR LOS DATOS (SOLO LOS VALORES DE INPUT)
X_scaled = scaler.fit_transform(X)

# Mostrar los primeros resultados
print("Inputs normalizados (primeras filas):")
print(X_scaled[:5])

Inputs normalizados (primeras filas):
[[ 1.03516957  1.10682993  1.03516957  1.10682993 -1.24199419 -1.2235944
   1.03516957 -1.2235944  -1.23155602]
 [ 1.03516957  1.10682993  1.03516957  1.10682993 -1.24199419 -1.2235944
  -1.23155602  1.10682993 -1.23155602]
 [ 1.03516957  1.10682993  1.03516957  1.10682993 -1.24199419 -1.2235944
  -1.23155602 -1.2235944   1.03516957]
 [ 1.03516957  1.10682993  1.03516957  1.10682993 -1.24199419 -1.2235944
  -1.23155602 -0.05838224 -0.09819323]
 [ 1.03516957  1.10682993  1.03516957  1.10682993 -1.24199419 -1.2235944
  -0.09819323 -1.2235944  -0.09819323]]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [28]:
# your code h
#VAMOS A DIVIDIR LOS DATOS NORMALIZADOS EN CONJUNTO DE ENTRENAMIENTO Y PRUEBA.

from sklearn.model_selection import train_test_split

# Divide en entrenamiento y prueba (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"Tamaño de entrenamiento: {X_train.shape}")
print(f"Tamaño de prueba: {X_test.shape}")

Tamaño de entrenamiento: (766, 9)
Tamaño de prueba: (192, 9)


In [ ]:
#CREAR EL SEQUENTIAL MODEL 
#Dense(32) → Primera capa oculta con 32 neuronas.
#ReLU → Función de activación que ayuda a aprender relaciones complejas.
#Capa final Dense(2) con softmax → porque tienes 2 clases.
#sparse_categorical_crossentropy → para clasificación entera (0 y 1) sin necesidad de codificar las etiquetas.

!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 33.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 35.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 35.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 43.7 MB/s eta 0:00:00


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 882 (3.45 KB)

 Trainable params: 882 (3.45 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Crear el modelo
model = Sequential()

# Capa de entrada + primera capa oculta
model.add(Dense(32, activation='relu', input_shape=(9,)))  # 9 entradas (9 casillas)

# Segunda capa oculta
model.add(Dense(16, activation='relu'))

# Capa de salida (2 clases → Player X gana o no)
model.add(Dense(2, activation='softmax'))  # Softmax para clasificación

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Mostrar resumen del modelo
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 32)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 882 (3.45 KB)

 Trainable params: 882 (3.45 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
#PARA ENTRENAR EL MODELO

history = model.fit(
    X_train, y_train,
    epochs=30,  # Puedes ajustar el número de épocas si quieres probar más
    batch_size=32,  # Tamaño del "bloque" de datos que procesa a la vez (32 es buen valor inicial)
    validation_split=0.1,  # Reservar 10% del train para validación
    verbose=1  # Muestra progreso
)

Epoch 1/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4680 - loss: 0.7240 - val_accuracy: 0.5195 - val_loss: 0.6760
Epoch 2/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5761 - loss: 0.6613 - val_accuracy: 0.6364 - val_loss: 0.6238
Epoch 3/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6923 - loss: 0.6128 - val_accuracy: 0.7273 - val_loss: 0.5845
Epoch 4/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7330 - loss: 0.5867 - val_accuracy: 0.7792 - val_loss: 0.5521
Epoch 5/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7554 - loss: 0.5531 - val_accuracy: 0.7922 - val_loss: 0.5210
Epoch 6/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7614 - loss: 0.5460 - val_accuracy: 0.8052 - val_loss: 0.4947
Epoch 7/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7818 - loss: 0.5188 - val_accuracy: 0.8052 - val_loss: 0.4717
Epoch 8/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7855 - loss: 0.4927 - val_accuracy: 0.8052 - val_loss:

In [33]:
#EVALUAR EL MODELO CON LOS DATOS DE PRUEBA
# Evaluar el modelo en los datos de prueba
#RESULTADO: TEST LOSS IS 10% ESTO ES EL PROMEDIO DE ERROR EN LAS PREDICCIONES DEL MODELO
#           TEST ACCURACY 98% EL MODELO ACERTO CASI UN 98% SOBRE CONFIGURACIONES NUEVAS DE TIC TAC TOC QUE NUNCA HABIA VISTO. 

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

print(f"\nTest Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9717 - loss: 0.1023

Test Loss: 0.10258904099464417
Test Accuracy: 0.9791666865348816


In [37]:
#GUARDAR EL MODELO 
model.save('tic-tac-toe.keras')



## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [38]:
# your code here

from tensorflow.keras.models import load_model

# Cargar el modelo
model_loaded = load_model('tic-tac-toe.keras')

In [39]:
import numpy as np

# Seleccionar 5 filas aleatorias del test set
random_indices = np.random.choice(len(X_test), size=5, replace=False)
sample_data = X_test[random_indices]
sample_labels = y_test.values[random_indices]

# Hacer predicciones
predictions = model_loaded.predict(sample_data)

# Mostrar resultados
for i in range(len(sample_data)):
    predicted_class = np.argmax(predictions[i])
    actual_class = sample_labels[i]
    print(f"\nMuestra {i+1}:")
    print(f"Predicción: {predicted_class} - Real: {actual_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Muestra 1:
Predicción: 1 - Real: 1

Muestra 2:
Predicción: 0 - Real: 0

Muestra 3:
Predicción: 1 - Real: 1

Muestra 4:
Predicción: 1 - Real: 0

Muestra 5:
Predicción: 0 - Real: 0


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [41]:
# your code here

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Crear un modelo mejorado con más capas
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(9,)))  # Capa oculta más grande
model.add(Dense(32, activation='relu'))                    # Segunda capa oculta
model.add(Dense(16, activation='relu'))                    # Tercera capa oculta (nueva)
model.add(Dense(2, activation='softmax'))                  # Capa de salida (2 clases)

# Ajustar el learning rate en el optimizador Adam
optimizer = Adam(learning_rate=0.001)  # Puedes probar con otros valores como 0.0005 o 0.005

# Compilar el modelo
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Entrenar el modelo con más épocas
history = model.fit(
    X_train, y_train,
    epochs=50,  # Más épocas para mejorar
    batch_size=32,
    validation_split=0.1
)

# Evaluar en el test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"\nTest Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6173 - loss: 0.6350 - val_accuracy: 0.7532 - val_loss: 0.5268
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7043 - loss: 0.5602 - val_accuracy: 0.8052 - val_loss: 0.4907
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7596 - loss: 0.5226 - val_accuracy: 0.8571 - val_loss: 0.4583
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8165 - loss: 0.4537 - val_accuracy: 0.8312 - val_loss: 0.4207
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8686 - loss: 0.4051 - val_accuracy: 0.8831 - val_loss: 0.3623
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9081 - loss: 0.3354 - val_accuracy: 0.9091 - val_loss: 0.3261
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9333 - loss: 0.2904 - val_accuracy: 0.9221 - val_loss: 0.2498
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9566 - loss: 0.2270 - val_accuracy: 0.9610 - val_loss:

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
#ADDING MORE HIDDEN LAYERS
#INCREASING THE NUMBER OF NEURONS IN EACH LAYER IMPROVED THE MODEL 